In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import math

# Tải dữ liệu
file_path =  r'C:\Users\Admin\Desktop\HK1_2024\AI\final\dt_data.csv'
data = pd.read_csv(file_path)

# Loại bỏ các cột không cần thiết hoặc không có tên
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
#check duplicate
duplicates = data[data.duplicated()]
if not duplicates.empty:
    print("Duplicates found:")
    print(duplicates)
else:
    print("No duplicates found.")
# Các hàm tính toán Entropy, Average Entropy và Information Gain
def entropy(column):
    counts = np.bincount(column)
    probabilities = counts / len(column)
    entropy = -np.sum([p * math.log2(p) for p in probabilities if p > 0])
    return entropy

def info_gain(data, split_attribute_name, target_name="Rank"):
    total_entropy = entropy(data[target_name])
    
    vals, counts = np.unique(data[split_attribute_name], return_counts=True)
    weighted_entropy = np.sum([(counts[i] / np.sum(counts)) * 
                               entropy(data.where(data[split_attribute_name] == vals[i]).dropna()[target_name])
                               for i in range(len(vals))])
    
    information_gain = total_entropy - weighted_entropy
    return information_gain

# Nhập tên thuộc tính từ người dùng
attribute_name = input("Enter the name of the score attribute (e.g., Q1, Q2, ...): ")

# Tính Entropy, Average Entropy và Information Gain
H = entropy(data[attribute_name])
AE = np.mean([entropy(data[column]) for column in data.columns if column.startswith('Q')])
IG = info_gain(data, attribute_name)

# Hiển thị kết quả
print(f"Entropy (H) of {attribute_name}: {H}")
print(f"Average Entropy (AE) of {attribute_name}: {AE}")
print(f"Information Gain (IG) of {attribute_name}: {IG}")

# Chọn các cột Q1 - Q9 làm feature và Rank làm target
X = data[['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9']]
y = data['Rank']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Khởi tạo và huấn luyện mô hình Decision Tree
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Dự đoán và đánh giá mô hình
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the Decision Tree model: {accuracy}")

In [ ]:
plt.figure(figsize=(20,10))
plot_tree(clf, feature_names=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9'], class_names=np.unique(y).astype(str), filled=True)
plt.show()
